In [1]:
import pandas as pd
import numpy as np
from pandas.core.reshape.merge import merge
from pandas.core.groupby.grouper import get_grouper

In [7]:
driver_standings = pd.read_csv('data/driver_standings.csv')
drivers = pd.read_csv('data/drivers.csv')
races = pd.read_csv('data/races.csv',parse_dates=['year'])
constructors = pd.read_csv("data/constructors.csv")
results = pd.read_csv('data/results.csv')

In [8]:
results_f=pd.merge(results, constructors, on='constructorId', how='inner')


race_wins = drivers.merge(results_f,left_on='driverId',right_on='driverId',how = 'left')
race_wins.head()
# race_wins['full_name'] = race_wins['forename'] + ' ' + race_wins['surname']

# race_wins = race_wins[['full_name','position', 'name']]

,driverId,driverRef,number_x,code,forename,surname,dob,nationality_x,url_x,resultId,...,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId,constructorRef,name,nationality_y,url_y
0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,1,...,5690616,39,2,1:27.452,218.300,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren
1,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,27,...,5525103,53,3,1:35.462,209.033,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren
2,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,57,...,\N,25,19,1:35.520,203.969,11,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren
3,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,69,...,5903238,20,3,1:22.017,204.323,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren
4,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,90,...,5213230,31,2,1:26.529,222.085,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren


In [9]:
race_wins['full_name'] = race_wins['forename'] + ' ' + race_wins['surname']

race_wins = race_wins[['full_name','position', 'name']]

race_wins

,full_name,position,name
0,Lewis Hamilton,1,McLaren
1,Lewis Hamilton,5,McLaren
2,Lewis Hamilton,13,McLaren
3,Lewis Hamilton,3,McLaren
4,Lewis Hamilton,2,McLaren
...,...,...,...
26075,Logan Sargeant,\N,Williams
26076,Logan Sargeant,13,Williams
26077,Logan Sargeant,11,Williams
26078,Logan Sargeant,\N,Williams


In [10]:
# Assuming your DataFrame is named race_wins
race_wins = race_wins[pd.to_numeric(race_wins['position'], errors='coerce').notnull()]
race_wins['position'] = race_wins['position'].astype(int)
race_wins

/var/folders/f6/jcyyqyjj3xvbt_h8_sjjvr4r0000gn/T/ipykernel_18295/619006296.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  race_wins['position'] = race_wins['position'].astype(int)


,full_name,position,name
0,Lewis Hamilton,1,McLaren
1,Lewis Hamilton,5,McLaren
2,Lewis Hamilton,13,McLaren
3,Lewis Hamilton,3,McLaren
4,Lewis Hamilton,2,McLaren
...,...,...,...
26073,Logan Sargeant,18,Williams
26074,Logan Sargeant,20,Williams
26076,Logan Sargeant,13,Williams
26077,Logan Sargeant,11,Williams


In [11]:
# highest_rw = race_wins[race_wins['position'] == 1.0].groupby('full_name').count().sort_values(
#     'position', ascending = False).reset_index()
# highest_rw.head(5)

# highest_rw=race_wins[race_wins['position'] == '1.0']

# race_wins.info()

# Replace 'N/A' with 20 in the entire DataFrame
race_wins_filled = race_wins.fillna(20)

# Convert 'position' column to integers
race_wins_filled['position'] = race_wins_filled['position'].astype(int)
race_wins

,full_name,position,name
0,Lewis Hamilton,1,McLaren
1,Lewis Hamilton,5,McLaren
2,Lewis Hamilton,13,McLaren
3,Lewis Hamilton,3,McLaren
4,Lewis Hamilton,2,McLaren
...,...,...,...
26073,Logan Sargeant,18,Williams
26074,Logan Sargeant,20,Williams
26076,Logan Sargeant,13,Williams
26077,Logan Sargeant,11,Williams


In [12]:

highest_rw = race_wins[race_wins['position'] == 1.0].groupby('full_name').agg({'name': 'first', 'position': 'count'}).sort_values(
    'position', ascending=False).reset_index()
highest_rw.head(5)


,full_name,name,position
0,Lewis Hamilton,McLaren,103
1,Michael Schumacher,Ferrari,91
2,Sebastian Vettel,Toro Rosso,53
3,Alain Prost,McLaren,51
4,Max Verstappen,Red Bull,45


In [13]:
driver_quali = results.merge(drivers,left_on = 'driverId',right_on='driverId',how = 'left')
driver_quali['full_name'] = driver_quali['forename'] + ' ' + driver_quali['surname']
driver_quali = driver_quali[['full_name','grid','position']]


quali_wins = driver_quali[driver_quali['grid'] == 1].groupby('full_name')['grid'].sum().reset_index(
                                        name = 'grid poles').sort_values('grid poles',ascending = False)

quali_wins = quali_wins.reset_index(col_level=0)
quali_wins.drop('index',axis = 1,inplace = True)
quali_wins.head(10)

,full_name,grid poles
0,Lewis Hamilton,104
1,Michael Schumacher,68
2,Ayrton Senna,65
3,Sebastian Vettel,57
4,Jim Clark,34
5,Alain Prost,33
6,Nigel Mansell,32
7,Nico Rosberg,30
8,Juan Fangio,29
9,Max Verstappen,27


In [17]:
racexpole = highest_rw.merge(quali_wins,on = 'full_name',how = 'left')
# racexpole = racexpole[racexpole['grid poles'] > 10]                    #setting minimum grid poles as > 10 
racexpole.dropna(inplace=True) 
# racexpole.to_csv("data/racexpole.csv")
racexpole.head(10)

,full_name,name,position,grid poles
0,Lewis Hamilton,McLaren,103,104.0
1,Michael Schumacher,Ferrari,91,68.0
2,Sebastian Vettel,Toro Rosso,53,57.0
3,Alain Prost,McLaren,51,33.0
4,Max Verstappen,Red Bull,45,27.0
5,Ayrton Senna,McLaren,41,65.0
6,Fernando Alonso,McLaren,32,22.0
7,Nigel Mansell,Williams,31,32.0
8,Jackie Stewart,Tyrrell,27,17.0
9,Niki Lauda,McLaren,25,24.0


In [18]:
racexpole.rename(columns={'position':'race poles'},inplace=True)
racexpole['grid poles'] = racexpole['grid poles'].astype(int)
racexpole['racexgrid'] = (racexpole['race poles']/racexpole['grid poles']).round(2)
racexpole = racexpole.sort_values('racexgrid',ascending=False).reset_index(col_level=0)
racexpole.drop('index',axis = 1,inplace = True)
racexpole.to_csv('data/racexpole.csv')
racexpole.head(15)


,full_name,name,race poles,grid poles,racexgrid
0,Denny Hulme,McLaren,8,1,8.00
1,Jody Scheckter,Ferrari,10,3,3.33
2,Gilles Villeneuve,Ferrari,6,2,3.00
3,Thierry Boutsen,Williams,3,1,3.00
4,Daniel Ricciardo,McLaren,8,3,2.67
5,Michele Alboreto,Ferrari,5,2,2.50
6,John Watson,McLaren,5,2,2.50
7,Emerson Fittipaldi,McLaren,14,6,2.33
8,Sergio Pérez,Red Bull,6,3,2.00
9,Bill Vukovich,Kurtis Kraft,2,1,2.00


In [57]:
req=racexpole[racexpole['name']== 'Ferrari']
req

,full_name,name,race poles,grid poles,racexgrid
4,Michael Schumacher,Ferrari,91,68,1.34
15,Alberto Ascari,Ferrari,13,14,0.93
18,Juan Fangio,Ferrari,24,29,0.83
20,Rubens Barrichello,Ferrari,11,14,0.79
26,Felipe Massa,Ferrari,11,16,0.69
28,Mario Andretti,Ferrari,12,18,0.67
30,Jacky Ickx,Ferrari,8,13,0.62
34,Charles Leclerc,Ferrari,5,20,0.25
